In [1]:
import logging
import os
import sched
import sys
import time
import pandas as pd
from datetime import datetime
from models.PyCryptoBot import PyCryptoBot
from models.AppState import AppState
from models.Trading import TechnicalAnalysis
from models.TradingAccount import TradingAccount
from models.helper.LogHelper import Logger

app = PyCryptoBot()
account = TradingAccount(app)
technical_analysis = None
state = AppState(app, account)
state.initLastAction()

config = {}
trading_historical_data = {}
dataframe_dict = []

pd.set_option('display.max_columns', None)
# Logger.configure(consoleloglevel="ERROR")

def getAction(now: datetime = datetime.today().strftime('%Y-%m-%d %H:%M:%S'), app: PyCryptoBot = None, price: float = 0,
              df: pd.DataFrame = pd.DataFrame(), df_last: pd.DataFrame = pd.DataFrame(), last_action: str = 'WAIT') -> str:
    ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
    macdgtsignal = bool(df_last['macdgtsignal'].values[0])
    goldencross = bool(df_last['goldencross'].values[0])
    obv_pc = float(df_last['obv_pc'].values[0])
    elder_ray_buy = bool(df_last['eri_buy'].values[0])
    ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
    macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
    ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
    macdltsignal = bool(df_last['macdltsignal'].values[0])
    rri_buy = bool(df_last['rri_buy'].values[0])
    rsi = bool(df_last['rsi14'].values[0])

    action = '' 

    # criteria for a buy signal
    if ema12gtema26co is True \
            and (macdgtsignal is True or app.disableBuyMACD()) \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26co: {ema12gtema26co}')

        if not app.disableBuyMACD():
            Logger.debug(f'macdgtsignal: {macdgtsignal}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif ema12gtema26 is True \
            and macdgtsignalco is True \
            and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'ema12gtema26: {ema12gtema26}')
        Logger.debug(f'macdgtsignalco: {macdgtsignalco}')

        if not app.disableBullOnly():
            Logger.debug(f'goldencross: {goldencross}')

        if not app.disableBuyOBV():
            Logger.debug(f'obv_pc: {obv_pc} > -5')

        if not app.disableBuyElderRay():
            Logger.debug(f'elder_ray_buy: {elder_ray_buy}')

        Logger.debug(f'last_action: {last_action}')

    elif rri_buy is True and (goldencross is True or app.disableBullOnly()) \
            and (obv_pc > -5 or app.disableBuyOBV()) \
            and (elder_ray_buy is True or app.disableBuyElderRay()) \
            and last_action != 'BUY':

        action = 'BUY'

        Logger.debug('*** Buy Signal ***')
        Logger.debug(f'rri_buy: {rri_buy}')

    # criteria for a sell signal
    elif ema12ltema26co is True \
            and (macdltsignal is True or app.disableBuyMACD()) \
            and last_action not in ['', 'SELL']:

        action = 'SELL'

        Logger.debug('*** Sell Signal ***')
        Logger.debug(f'ema12ltema26co: {ema12ltema26co}')
        Logger.debug(f'macdltsignal: {macdltsignal}')
        Logger.debug(f'last_action: {last_action}')

    # anything other than a buy or sell, just wait
    else:
        action = 'WAIT'

    # if disabled, do not buy within 3% of the dataframe close high
    if last_action == 'SELL' and app.disableBuyNearHigh() is True and (price > (df['close'].max() * 0.97)):
        log_text = str(now) + ' | ' + app.getMarket() + ' | ' + \
            app.printGranularity() + ' | Ignoring Buy Signal (price ' + str(price) + ' within 3% of high ' + str(
            df['close'].max()) + ')'
        Logger.warning(log_text)

        action = 'WAIT'

    return action


def getInterval(df: pd.DataFrame = pd.DataFrame(), app: PyCryptoBot = None, iterations: int = 0) -> pd.DataFrame:
    
    return df.iloc[iterations - 1:iterations]
    

def find_high_low_interval(df, start, next_x, price):
    df_clipped = df.iloc[start:start+next_x]
    high_pct = round(((df_clipped["high"].max()/price) - 1) * 100, 2)
    low_pct = round(((df_clipped["low"].min()/price) - 1) * 100, 2)
    return (high_pct, low_pct)

In [2]:
def detect_historical_buys(coin_pair):
    state.iterations = 0
    print("calculating ", coin_pair)
    if coin_pair not in trading_historical_data:
        trading_historical_data[coin_pair] = app.getHistoricalData(coin_pair, 3600)
    trading_data = trading_historical_data[coin_pair]
    trading_dataCopy = trading_data.copy()
    technical_analysis = TechnicalAnalysis(trading_dataCopy)
    technical_analysis.addAll()
    df = technical_analysis.getDataFrame()
    while (state.iterations < 300):
        state.iterations = state.iterations + 1
        df_last = getInterval(df, app, state.iterations)
        if len(df_last) > 0:
            now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            price = float(df_last['close'].values[0])
            if price < 0.0001:
                continue
                raise Exception(app.getMarket() + ' is unsuitable for trading, quote price is less than 0.0001!')

            state.action = getAction(now, app, price, df, df_last, state.last_action)
            if state.action=="BUY":
                # technical indicators
                ema12gtema26 = bool(df_last['ema12gtema26'].values[0])
                ema12gtema26co = bool(df_last['ema12gtema26co'].values[0])
                goldencross = bool(df_last['goldencross'].values[0])
                macdgtsignal = bool(df_last['macdgtsignal'].values[0])
                macdgtsignalco = bool(df_last['macdgtsignalco'].values[0])
                ema12ltema26 = bool(df_last['ema12ltema26'].values[0])
                ema12ltema26co = bool(df_last['ema12ltema26co'].values[0])
                macdltsignal = bool(df_last['macdltsignal'].values[0])
                macdltsignalco = bool(df_last['macdltsignalco'].values[0])
                obv = float(df_last['obv'].values[0])
                obv_pc = float(df_last['obv_pc'].values[0])
                elder_ray_buy = bool(df_last['eri_buy'].values[0])
                elder_ray_sell = bool(df_last['eri_sell'].values[0])
                rri = df_last['rri'].values[0]
                last_3_rri_buy = df_last['last_3_rri_buy'].values[0]
                last_5_rri_buy = df_last['last_5_rri_buy'].values[0]
                last_3_rri_sell = df_last['last_3_rri_sell'].values[0]
                last_5_rri_sell = df_last['last_5_rri_sell'].values[0]
                rri_buy = bool(df_last['rri_buy'].values[0])
                rri_sell = df_last['rri_sell'].values[0]
                rsi = df_last['rsi14'].values[0]
                rsi_last = df_last['rsi_last'].values[0]
                rsi_prev = df_last['rsi_prev'].values[0]
                stochrsi14 = float(df_last['stochrsi14'].values[0])
                williamsr14 = float(df_last['williamsr14'].values[0])

                hammer = bool(df_last['hammer'].values[0])
                inverted_hammer = bool(df_last['inverted_hammer'].values[0])
                hanging_man = bool(df_last['hanging_man'].values[0])
                shooting_star = bool(df_last['shooting_star'].values[0])
                three_white_soldiers = bool(df_last['three_white_soldiers'].values[0])
                three_black_crows = bool(df_last['three_black_crows'].values[0])
                morning_star = bool(df_last['morning_star'].values[0])
                evening_star = bool(df_last['evening_star'].values[0])
                three_line_strike = bool(df_last['three_line_strike'].values[0])
                abandoned_baby = bool(df_last['abandoned_baby'].values[0])
                morning_doji_star = bool(df_last['morning_doji_star'].values[0])
                evening_doji_star = bool(df_last['evening_doji_star'].values[0])
                two_black_gapping = bool(df_last['two_black_gapping'].values[0])

                # if simulation interations < 200 set goldencross to true
                if app.isSimulation() and state.iterations < 200:
                    goldencross = True
                buy_score = 0
                if ema12gtema26: buy_score += 3
                if ema12gtema26co: buy_score += 3
                if macdgtsignal: buy_score += 3
                if goldencross: buy_score += 2
                if macdgtsignalco: buy_score +=3
                if obv_pc > -5: buy_score += 3
                if elder_ray_buy: buy_score += 2
                if not elder_ray_sell: buy_score += 2
                if rri_buy and (ema12gtema26 or ema12gtema26co) and (ema12gtema26co or macdgtsignalco):
                    buy_score += 10
                elif rri_buy and (ema12gtema26 and macdgtsignal):
                    buy_score += 8
                elif rri_buy and (ema12gtema26 or macdgtsignal):
                    buy_score += 6
                elif rri_buy:
                    buy_score += 4
                candle_stick_score = 0
                # weak up
                if hammer is True: candle_stick_score += 1
                if inverted_hammer is True: candle_stick_score += 1
                # weak down
                if shooting_star is True: candle_stick_score -= 1
                if hanging_man is True: candle_stick_score -= 1
                # strong up
                if three_white_soldiers is True: candle_stick_score += 2
                if morning_star is True: candle_stick_score += 2
                # strong down
                if three_black_crows is True: candle_stick_score -= 2
                if evening_star is True: candle_stick_score -= 2
                # reliable up
                if three_line_strike is True: candle_stick_score += 3
                if abandoned_baby is True: candle_stick_score += 3
                if morning_doji_star is True: candle_stick_score += 3
                # reliable down
                if evening_doji_star is True: candle_stick_score -= 3
                if two_black_gapping is True: candle_stick_score -= 3
                sarima_1 = None
                sarima_1_margin = None
                sarima_1_dt = None
                sarima_2 = None
                sarima_2_margin = None
                sarima_2_dt = None
                sarima_3 = None
                sarima_3_margin = None
                sarima_3_dt = None
                sarima_6 = None
                sarima_6_margin = None
                sarima_6_dt = None
                splitted_df = df.iloc[0:state.iterations]
                prediction_minutes = int(app.getGranularity() / 60)
                if (prediction_minutes<len(splitted_df)):
                    sarima = technical_analysis.customSARIMAPrediction(prediction_minutes,
                                                                             splitted_df)
                    sarima_1 = sarima[1]
                    sarima_1_margin = ((sarima_1 / price) - 1) * 100
                    sarima_1_dt = sarima[0]
                if (prediction_minutes*2<len(splitted_df)):
                    sarima = technical_analysis.customSARIMAPrediction(prediction_minutes*2,
                                                                             splitted_df)
                    sarima_2 = sarima[1]
                    sarima_2_margin = ((sarima_2 / price) - 1) * 100
                    sarima_2_dt = sarima[0]
                if (prediction_minutes*3<len(splitted_df)):
                    sarima = technical_analysis.customSARIMAPrediction(prediction_minutes*3,
                                                                             splitted_df)
                    sarima_3 = sarima[1]
                    sarima_3_margin = ((sarima_3 / price) - 1) * 100
                    sarima_3_dt = sarima[0]
                if (prediction_minutes*6<len(splitted_df)):
                    sarima = technical_analysis.customSARIMAPrediction(prediction_minutes*6,
                                                                             splitted_df)
                    sarima_6 = sarima[1]
                    sarima_6_margin = ((sarima_6 / price) - 1) * 100
                    sarima_6_dt = sarima[0]
                
                
                one_hr_high, one_hr_low = find_high_low_interval(df, state.iterations, 1, price)
                three_hr_high, three_hr_low = find_high_low_interval(df, state.iterations, 3, price)
                six_hr_high, six_hr_low = find_high_low_interval(df, state.iterations, 6, price)
                nine_hr_high, nine_hr_low = find_high_low_interval(df, state.iterations, 9, price)
                twelve_hr_high, twelve_hr_low = find_high_low_interval(df, state.iterations, 12, price)
                
                three_six_high, three_six_low = find_high_low_interval(df, state.iterations+3, 6, price)
                six_12_high, six_12_low = find_high_low_interval(df, state.iterations+6, 12, price)
                dataframe_dict.append({"action": state.action,
                                       "coin_pair": coin_pair,
                                       "timestamp": df_last['date'][0].isoformat(),
                                       "buy_score": buy_score, 
                                       "candle_stick_score":candle_stick_score,
                                       "granularity": app.getGranularity(),
                                       "price": price,
                                       "one_hr_high": one_hr_high, 
                                       "one_hr_low": one_hr_low, 
                                       "three_hr_high": three_hr_high, 
                                       "three_hr_low": three_hr_low, 
                                       "three_six_high": three_six_high,
                                       "three_six_low": three_six_low, 
                                       "six_12_high": six_12_high, 
                                       "six_12_low": six_12_low, 
                                       "six_hr_high": six_hr_high, 
                                       "six_hr_low": six_hr_low,
                                       "nine_hr_high": nine_hr_high, 
                                       "nine_hr_low": nine_hr_low, 
                                       "twelve_hr_high": twelve_hr_high, 
                                       "twelve_hr_low": twelve_hr_low,
                                       "ema12gtema26": ema12gtema26,
                                       "ema12gtema26co": ema12gtema26co,
                                       "goldencross": goldencross,
                                       "macdgtsignal": macdgtsignal, 
                                       "macdgtsignalco": macdgtsignalco, 
                                       "obv": obv, 
                                       "obv_pc": obv_pc, 
                                       "elder_ray_buy": elder_ray_buy,
                                       "elder_ray_sell": elder_ray_sell, 
                                       "rri": rri, 
                                       "rri_buy": rri_buy, 
                                       "rri_sell": rri_sell, 
                                       "last_3_rri_buy": last_3_rri_buy, 
                                       "last_5_rri_buy": last_5_rri_buy, 
                                       "last_3_rri_sell": last_3_rri_sell, 
                                       "last_5_rri_sell": last_5_rri_sell, 
                                       "rsi": rsi,
                                       "rsi_last": rsi_last,
                                       "rsi_prev": rsi_prev,
                                       "stochrsi14": stochrsi14,
                                       "williamsr14": williamsr14,
                                       "sarima_1": sarima_1,
                                       "sarima_1_margin": sarima_1_margin,
                                       "sarima_1_dt": sarima_1_dt,
                                       "sarima_2": sarima_2,
                                       "sarima_2_margin": sarima_2_margin,
                                       "sarima_2_dt": sarima_2_dt,
                                       "sarima_3": sarima_3,
                                       "sarima_3_margin": sarima_3_margin,
                                       "sarima_3_dt": sarima_3_dt,
                                       "sarima_6": sarima_3,
                                       "sarima_6_margin": sarima_6_margin,
                                       "sarima_6_dt": sarima_6_dt,
                                      })
    

In [3]:
%%time
binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB', 'XLMBNB', 'WABIBNB', 'LTCBNB', 'WAVESBNB', 'ICXBNB',
                          'BLZBNB', 'ZILBNB', 'ONTBNB', 'WANBNB', 'ADABNB', 'ZENBNB', 'EOSBNB', 'THETABNB', 'XRPBNB',
                          'ENJBNB', 'TRXBNB', 'ETCBNB', 'SCBNB', 'MFTBNB', 'VETBNB', 'RVNBNB', 'MITHBNB', 'BTTBNB',
                          'HOTBNB', 'FETBNB', 'XMRBNB', 'ZECBNB', 'IOSTBNB', 'CELRBNB', 'DASHBNB', 'MATICBNB',
                          'ATOMBNB', 'ONEBNB', 'FTMBNB', 'ALGOBNB', 'ANKRBNB', 'WINBNB', 'COSBNB', 'COCOSBNB',
                          'PERLBNB', 'CHZBNB', 'BANDBNB', 'XTZBNB', 'HBARBNB', 'STXBNB', 'KAVABNB', 'ARPABNB', 'BCHBNB',
                          'TROYBNB', 'FTTBNB', 'OGNBNB', 'WRXBNB', 'MBLBNB', 'COTIBNB', 'SOLBNB', 'CTSIBNB', 'CHRBNB',
                          'STMXBNB', 'IQBNB', 'DGBBNB', 'SXPBNB', 'SNXBNB', 'VTHOBNB', 'MKRBNB', 'RUNEBNB', 'FIOBNB',
                          'AVABNB', 'YFIBNB', 'SRMBNB', 'ANTBNB', 'CRVBNB', 'SANDBNB', 'OCEANBNB', 'NMRBNB',
                          'DOTBNB', 'LUNABNB', 'RSRBNB', 'PAXGBNB', 'WNXMBNB', 'SUSHIBNB', 'YFIIBNB', 'KSMBNB',
                          'EGLDBNB', 'DIABNB', 'BELBNB', 'WINGBNB', 'SWRVBNB', 'CREAMBNB', 'UNIBNB', 'AVAXBNB',
                          'BAKEBNB', 'BURGERBNB', 'CAKEBNB', 'SPARTABNB', 'XVSBNB', 'ALPHABNB', 'AAVEBNB', 'NEARBNB',
                          'FILBNB', 'INJBNB', 'CTKBNB', 'KP3RBNB', 'AXSBNB', 'HARDBNB', 'UNFIBNB', 'PROMBNB', 'BIFIBNB',
                          'ICPBNB', 'BTCUSDT', 'ETHUSDT', 'BCCUSDT', 'NEOUSDT', 'LTCUSDT', 'QTUMUSDT', 'ADAUSDT',
                       'XRPUSDT', 'EOSUSDT', 'IOTAUSDT', 'XLMUSDT', 'ONTUSDT', 'TRXUSDT', 'ETCUSDT',
                       'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'BCHABCUSDT', 'BCHSVUSDT',
                       'LINKUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT',
                       'FETUSDT', 'BATUSDT', 'XMRUSDT', 'ZECUSDT', 'IOSTUSDT', 'CELRUSDT', 'DASHUSDT', 'NANOUSDT',
                       'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'ATOMUSDT', 'TFUELUSDT', 'ONEUSDT',
                       'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DOGEUSDT', 'DUSKUSDT', 'ANKRUSDT',
                       'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT',
                       'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT',
                       'BANDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT',
                       'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'BCHUSDT',
                       'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT',
                       'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'WRXUSDT', 'BTSUSDT', 'LSKUSDT', 'BNTUSDT', 'LTOUSDT',
                       'EOSBULLUSDT', 'EOSBEARUSDT', 'XRPBULLUSDT', 'XRPBEARUSDT', 'STRATUSDT', 'AIONUSDT', 'MBLUSDT',
                       'COTIUSDT', 'STPTUSDT', 'WTCUSDT', 'DATAUSDT', 'XZCUSDT',
                       'SOLUSDT', 'CTSIUSDT', 'HIVEUSDT', 'CHRUSDT', 'GXSUSDT', 'ARDRUSDT',
                       'LENDUSDT', 'MDTUSDT', 'STMXUSDT', 'KNCUSDT', 'REPUSDT', 'LRCUSDT', 'PNTUSDT', 'COMPUSDT',
                       'BKRWUSDT', 'SCUSDT', 'ZENUSDT', 'SNXUSDT', 'VTHOUSDT', 'DGBUSDT', 'GBPUSDT', 'SXPUSDT',
                       'MKRUSDT', 'DCRUSDT', 'STORJUSDT', 'MANAUSDT', 'YFIUSDT', 'BALUSDT', 'BLZUSDT', 'IRISUSDT', 'KMDUSDT',
                       'SRMUSDT', 'ANTUSDT', 'CRVUSDT', 'SANDUSDT', 'OCEANUSDT', 'NMRUSDT', 'DOTUSDT',
                       'LUNAUSDT', 'RSRUSDT', 'WNXMUSDT', 'TRBUSDT', 'BZRXUSDT', 'SUSHIUSDT', 'YFIIUSDT',
                       'KSMUSDT', 'EGLDUSDT', 'DIAUSDT', 'RUNEUSDT', 'FIOUSDT', 'UMAUSDT', 'BELUSDT',
                       'WINGUSDT', 'UNIUSDT', 'NBSUSDT', 'OXTUSDT', 'SUNUSDT', 'AVAXUSDT',
                       'HNTUSDT', 'FLMUSDT', 'ORNUSDT', 'UTKUSDT', 'XVSUSDT', 'ALPHAUSDT',
                       'AAVEUSDT', 'NEARUSDT', 'FILUSDT',
                        'INJUSDT', 'AUDIOUSDT', 'CTKUSDT',
                       'AKROUSDT', 'AXSUSDT', 'HARDUSDT', 'DNTUSDT', 'STRAXUSDT', 'UNFIUSDT', 'ROSEUSDT', 'AVAUSDT',
                       'XEMUSDT', 'SKLUSDT', 'SUSDUSDT',
                        'GRTUSDT', 'JUVUSDT', 'PSGUSDT', '1INCHUSDT', 'REEFUSDT', 'OGUSDT',
                       'ATMUSDT', 'ASRUSDT', 'CELOUSDT', 'RIFUSDT', 'BTCSTUSDT', 'TRUUSDT', 'CKBUSDT', 'TWTUSDT',
                       'FIROUSDT', 'LITUSDT', 'SFPUSDT', 'DODOUSDT', 'CAKEUSDT', 'ACMUSDT', 'BADGERUSDT', 'FISUSDT',
                       'OMUSDT', 'PONDUSDT', 'DEGOUSDT', 'ALICEUSDT', 'LINAUSDT', 'PERPUSDT', 'RAMPUSDT', 'SUPERUSDT',
                       'CFXUSDT', 'EPSUSDT', 'AUTOUSDT', 'TKOUSDT', 'PUNDIXUSDT', 'TLMUSDT', 'BTGUSDT', 'MIRUSDT', 'BARUSDT', 'FORTHUSDT', 'BAKEUSDT', 'BURGERUSDT',
                       'SLPUSDT', 'SHIBUSDT', 'ICPUSDT', 'ARUSDT', 'POLSUSDT', 'MDXUSDT', 'MASKUSDT', 'BTCBUSD', 'XRPBUSD', 'ETHBUSD', 'LTCBUSD', 'LINKBUSD', 'ETCBUSD', 'TRXBUSD',
                       'EOSBUSD', 'XLMBUSD', 'ADABUSD', 'BCHBUSD', 'QTUMBUSD', 'VETBUSD', 'ICXBUSD',
                       'BNTBUSD', 'ATOMBUSD', 'DASHBUSD', 'NEOBUSD', 'WAVESBUSD', 'XTZBUSD', 'BATBUSD', 'ENJBUSD',
                       'NANOBUSD', 'ONTBUSD', 'RVNBUSD', 'ALGOBUSD', 'BTTBUSD', 'TOMOBUSD', 'XMRBUSD', 'ZECBUSD',
                       'DATABUSD', 'SOLBUSD', 'CTSIBUSD', 'HBARBUSD', 'MATICBUSD', 'WRXBUSD', 'ZILBUSD', 'KNCBUSD',
                       'LRCBUSD', 'IQBUSD', 'GBPBUSD', 'DGBBUSD', 'COMPBUSD', 'SXPBUSD', 'SNXBUSD', 'MKRBUSD',
                       'RUNEBUSD', 'MANABUSD', 'DOGEBUSD', 'ZRXBUSD', 'FIOBUSD', 'AVABUSD', 'IOTABUSD',
                       'BALBUSD', 'YFIBUSD', 'SRMBUSD', 'ANTBUSD', 'CRVBUSD', 'SANDBUSD', 'OCEANBUSD',
                       'NMRBUSD', 'DOTBUSD', 'LUNABUSD', 'IDEXBUSD', 'RSRBUSD', 'TRBBUSD', 'BZRXBUSD', 'SUSHIBUSD',
                       'YFIIBUSD', 'KSMBUSD', 'EGLDBUSD', 'DIABUSD', 'BELBUSD', 'SWRVBUSD', 'WINGBUSD', 'CREAMBUSD',
                       'UNIBUSD', 'AVAXBUSD', 'FLMBUSD', 'CAKEBUSD', 'XVSBUSD', 'ALPHABUSD', 'VIDTBUSD', 'AAVEBUSD',
                       'NEARBUSD', 'FILBUSD', 'INJBUSD', 'AERGOBUSD', 'ONEBUSD', 'AUDIOBUSD', 'CTKBUSD', 'KP3RBUSD',
                       'AXSBUSD', 'HARDBUSD', 'DNTBUSD', 'CVPBUSD', 'STRAXBUSD', 'FORBUSD', 'UNFIBUSD', 'FRONTBUSD',
                       'BCHABUSD', 'ROSEBUSD', 'SYSBUSD', 'HEGICBUSD', 'PROMBUSD', 'SKLBUSD', 'COVERBUSD', 'GHSTBUSD',
                       'DFBUSD', 'JUVBUSD', 'PSGBUSD', 'BTCSTBUSD', 'TRUBUSD', 'DEXEBUSD',
                       'CKBBUSD', 'TWTBUSD', 'LITBUSD', 'SFPBUSD', 'FXSBUSD', 'DODOBUSD', 'BAKEBUSD',
                       'UFTBUSD', '1INCHBUSD', 'BANDBUSD', 'GRTBUSD', 'IOSTBUSD', 'OMGBUSD', 'REEFBUSD', 'ACMBUSD',
                       'AUCTIONBUSD', 'PHABUSD', 'TVKBUSD', 'BADGERBUSD', 'FISBUSD', 'OMBUSD', 'PONDBUSD', 'DEGOBUSD',
                       'ALICEBUSD', 'CHZBUSD', 'BIFIBUSD', 'LINABUSD', 'PERPBUSD', 'RAMPBUSD', 'SUPERBUSD', 'CFXBUSD',
                       'XVGBUSD', 'EPSBUSD', 'AUTOBUSD', 'TKOBUSD', 'TLMBUSD', 'BTGBUSD', 'HOTBUSD', 'MIRBUSD',
                       'BARBUSD', 'FORTHBUSD', 'BURGERBUSD', 'SLPBUSD', 'SHIBBUSD', 'ICPBUSD', 'ARBUSD', 'POLSBUSD',
                       'MDXBUSD', 'MASKBUSD']
# binance_coin_pairs = ['WTCBNB', 'BATBNB', 'NEOBNB', 'IOTABNB']
dataframe_dict = []
for coin_pair in binance_coin_pairs:
    try:
        detect_historical_buys(coin_pair)
    except Exception as e:
        print(e)
buy_dataframe = pd.DataFrame(dataframe_dict)


calculating  WTCBNB
calculating  BATBNB
calculating  NEOBNB
calculating  IOTABNB
calculating  XLMBNB
calculating  WABIBNB


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


calculating  LTCBNB
calculating  WAVESBNB
calculating  ICXBNB


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


calculating  BLZBNB
calculating  ZILBNB


...

calculating  BURGERBUSD
calculating  SLPBUSD
calculating  SHIBBUSD
calculating  ICPBUSD
calculating  ARBUSD
calculating  POLSBUSD


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


calculating  MDXBUSD
calculating  MASKBUSD


/Users/aswin/projects/pycryptobot/venv/lib/python3.7/site-packages/statsmodels/tsa/statespace/sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'


CPU times: user 21h 26min 55s, sys: 3h 48min 6s, total: 1d 1h 15min 2s
Wall time: 4h 23min 47s


In [4]:
buy_dataframe

,action,coin_pair,timestamp,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,ema12gtema26,ema12gtema26co,goldencross,macdgtsignal,macdgtsignalco,obv,obv_pc,elder_ray_buy,elder_ray_sell,rri,rri_buy,rri_sell,last_3_rri_buy,last_5_rri_buy,last_3_rri_sell,last_5_rri_sell,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_1,sarima_1_margin,sarima_1_dt,sarima_2,sarima_2_margin,sarima_2_dt,sarima_3,sarima_3_margin,sarima_3_dt
0,BUY,WTCBNB,2021-06-02T13:00:00,11,0,3600,0.002120,2.88,0.52,2.88,-1.42,5.42,-0.66,7.12,1.60,5.42,-1.42,5.42,-1.42,5.85,-1.42,False,False,False,False,False,-17359.00,10.03,True,False,-0.852459,True,False,True,True,False,False,50.000000,50.000000,50.000000,0.500000,-50.000000,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
1,BUY,WTCBNB,2021-06-02T19:00:00,12,0,3600,0.002165,2.49,-0.46,2.49,-0.46,4.90,0.60,4.90,-2.68,3.65,-0.46,4.90,-0.46,4.90,-0.51,False,False,False,True,True,19300.30,-23.69,False,True,-1.000000,True,False,True,True,False,False,44.683392,50.000000,50.000000,0.000000,0.568966,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
2,BUY,WTCBNB,2021-06-02T21:00:00,12,0,3600,0.002184,0.05,-1.33,2.75,-1.33,3.98,-1.37,2.01,-3.57,3.98,-1.33,3.98,-1.37,3.98,-1.37,False,False,False,True,False,25717.80,-1.14,False,True,-0.833333,True,False,True,True,False,False,48.961119,53.632353,44.683392,0.478014,0.351724,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
3,BUY,WTCBNB,2021-06-03T02:00:00,16,0,3600,0.002237,-0.31,-2.10,-0.31,-3.71,-0.40,-5.81,-1.70,-5.86,-0.31,-3.71,-0.31,-5.81,-0.31,-5.81,True,True,False,True,False,29270.60,13.20,True,False,-0.181818,False,False,False,False,False,True,57.793028,51.274015,55.350505,1.000000,0.187845,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
4,BUY,WTCBNB,2021-06-03T03:00:00,9,0,3600,0.002190,1.46,-0.55,1.46,-1.64,1.74,-3.79,0.00,-3.84,1.74,-1.64,1.74,-3.79,1.74,-3.84,False,False,False,True,False,27679.10,-5.44,False,True,-1.000000,True,False,True,True,False,True,48.852053,57.793028,51.274015,0.317985,0.447514,NaN,NaN,None,NaN,NaN,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15938,BUY,MASKBUSD,2021-06-13T02:00:00,16,0,3600,4.379000,-0.16,-2.83,-0.16,-2.83,1.26,-2.83,5.05,-2.35,-0.16,-2.83,1.26,-2.83,1.26,-2.83,False,False,False,True,False,-76653.31,1.10,True,False,-0.969697,True,False,True,True,False,False,35.031988,36.466493,37.457281,0.000000,0.981203,4.399786,0.474671,2021-06-13 03:00:00,4.381730,0.062342,2021-06-13 04:00:00,4.390593,0.264751,2021-06-13 05:00:00
15939,BUY,MASKBUSD,2021-06-13T12:00:00,12,0,3600,4.313000,0.19,-0.65,0.77,-0.86,6.65,0.02,7.72,2.62,3.32,-0.86,6.65,-0.86,7.19,-0.86,False,False,False,True,False,-71817.99,1.37,False,True,-0.928571,True,False,True,True,False,False,38.376306,39.519095,41.314297,0.770394,0.768924,4.286243,-0.620383,2021-06-13 13:00:00,4.255461,-1.334086,2021-06-13 14:00:00,4.266046,-1.088661,2021-06-13 15:00:00
15940,BUY,MASKBUSD,2021-06-13T21:00:00,12,0,3600,4.484000,0.29,-0.47,3.10,-0.47,3.61,0.13,8.76,0.67,3.32,-0.47,3.61,-0.47,4.82,-0.47,True,True,False,True,False,-52787.06,9.36,False,True,-0.723214,False,False,False,False,True,True,55.271151,66.832875,57.312951,0.627808,0.358025,4.580164,2.144603,2021-06-13 22:00:00,4.566504,1.839973,2021-06-13 23:00:00,4.564657,1.798784,2021-06-14 00:00:00
15941,BUY,MASKBUSD,2021-06-14T00:00:00,17,0,3600,4.569000,1.40,0.15,1.40,-1.73,2.87,-1.20,6.74,-1.20,1.69,-1.73,2.87,-1.73,4.25,-1.73,True,False,False,True,False,-50941.59,4.04,False,True,-0.981481,True,False,True,True,True,True,59.654119,63.551267,54.704949,0.768903,0.155620,4.570520,0.033266,2021-06-14 01:00:00,4.543566,-0.556674,2021-06-14 02:00:00,4.562118,-0.150629,2021-06-14 03:00:00


In [5]:
buy_dataframe.describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_1,sarima_1_margin,sarima_2,sarima_2_margin,sarima_3,sarima_3_margin
count,15943.000000,15943.000000,15943.0,15943.000000,15879.000000,15879.000000,15879.000000,15879.000000,15671.000000,15671.000000,15437.000000,15437.000000,15879.000000,15879.000000,15879.000000,15879.000000,15879.000000,15879.000000,1.594300e+04,15943.000000,15931.000000,15943.000000,15943.000000,15669.000000,15943.000000,15943.000000,12872.000000,12872.000000,9621.000000,9621.000000,6275.000000,6275.000000
mean,12.427774,-0.002383,3600.0,320.727951,1.317442,-1.191743,2.315825,-2.078038,3.333870,-3.036269,4.632260,-4.327599,3.395867,-2.977907,4.198102,-3.668296,4.833710,-4.261362,-8.299903e+06,0.277449,-0.536571,47.287560,50.661659,49.010041,0.425002,-2.201259,318.777874,1.744369,281.693352,2.314333,300.503722,-0.514538
std,3.763260,0.337873,0.0,3239.519221,2.085551,1.370945,3.255968,2.134749,6.378091,4.064619,10.058856,5.710884,4.680569,2.906886,6.178606,3.527809,8.332731,4.087430,1.151079e+09,1092.086786,0.601915,11.208320,9.955801,9.948751,0.358246,11.571354,3219.901097,169.121274,2956.070899,206.143539,3086.173066,149.262294
min,4.000000,-5.000000,3600.0,0.000100,-1.480000,-24.010000,-1.430000,-24.640000,-12.690000,-78.180000,-13.370000,-78.740000,-1.300000,-24.640000,-1.300000,-78.180000,-1.300000,-78.740000,-4.591466e+10,-63815.260000,-1.000000,10.760062,12.382410,11.336908,0.000000,-50.000000,-0.042859,-5252.611534,-0.021995,-4281.618240,-0.022373,-10139.164926
25%,9.000000,0.000000,3600.0,0.062290,0.410000,-1.690000,0.680000,-2.840000,0.360000,-4.835000,0.460000,-7.260000,1.010000,-4.040000,1.230000,-5.120000,1.360000,-6.040000,-1.874278e+05,-7.500000,-0.893990,39.433370,44.804375,43.335926,0.010586,0.283354,0.062336,0.051529,0.061459,-0.054852,0.060251,-0.125073
50%,12.000000,0.000000,3600.0,0.789000,0.920000,-0.830000,1.500000,-1.470000,2.320000,-2.250000,3.140000,-3.200000,2.240000,-2.090000,2.750000,-2.580000,3.130000,-2.960000,2.896147e+03,0.000000,-0.810924,48.169160,50.868723,50.000000,0.437557,0.595975,0.759933,1.199905,0.709803,1.150519,0.698279,1.122161
75%,16.000000,0.000000,3600.0,6.435500,1.650000,-0.300000,2.900000,-0.610000,4.760000,-0.590000,6.710000,-0.670000,4.270000,-0.910000,5.080000,-1.160000,5.760000,-1.360000,6.806838e+05,7.665000,-0.651091,55.364912,56.962046,55.105160,0.722408,0.890230,6.282759,2.375440,6.057921,2.398833,6.064644,2.327623
max,28.000000,5.000000,3600.0,46991.950000,140.260000,2.110000,140.260000,1.720000,185.200000,39.760000,397.840000,51.990000,140.260000,1.720000,185.200000,1.720000,397.840000,1.720000,1.647208e+10,60413.430000,1.000000,89.267453,94.705191,90.348025,1.000000,1.000000,45411.979372,16994.051195,44854.896137,18812.818602,40629.670147,4230.002230


In [6]:
buy_dataframe.query("rsi<30").describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_1,sarima_1_margin,sarima_2,sarima_2_margin,sarima_3,sarima_3_margin
count,1063.000000,1063.000000,1063.0,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1.063000e+03,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,1063.000000,960.000000,960.000000,911.000000,911.000000,316.000000,316.000000
mean,8.439323,-0.035748,3600.0,336.519369,1.686914,-1.731872,2.691148,-2.679454,3.756557,-3.724186,5.707488,-5.117404,3.778852,-3.610602,4.494826,-4.585729,5.320837,-5.232352,-8.456646e+07,-57.654826,-0.848768,25.039066,30.201702,29.914280,0.111086,0.944664,330.134284,2.983068,343.919533,2.824062,34.356784,2.208917
std,2.355791,0.275561,0.0,3215.621220,1.403807,1.448536,2.339309,2.224266,4.224944,3.743394,7.682018,5.006013,3.304783,2.897633,3.881906,3.226783,5.172707,3.347977,1.880930e+09,1710.989575,0.074653,3.859906,4.861595,6.225699,0.203830,0.055008,3150.208835,2.260514,3196.394647,2.395351,214.219297,2.157537
min,4.000000,-3.000000,3600.0,0.000106,-0.630000,-8.290000,-0.580000,-15.440000,-5.270000,-16.730000,-9.660000,-22.870000,-0.290000,-15.440000,-0.270000,-16.730000,-0.090000,-22.550000,-4.352730e+10,-55253.900000,-1.000000,10.760062,12.382410,11.336908,0.000000,0.576375,0.000106,-2.414302,0.000151,-3.449603,0.000147,-6.394982
25%,7.000000,0.000000,3600.0,0.156050,0.730000,-2.460000,1.110000,-3.825000,1.145000,-6.350000,2.260000,-8.470000,1.610000,-5.510000,2.065000,-6.920000,2.445000,-7.485000,-1.254510e+06,-5.025000,-0.893661,22.665144,27.040686,25.768184,0.000000,0.923023,0.169590,1.271800,0.178191,0.994263,0.079864,0.729898
50%,9.000000,0.000000,3600.0,1.405900,1.380000,-1.570000,2.160000,-2.160000,3.180000,-3.210000,4.660000,-5.360000,3.160000,-2.730000,3.700000,-4.030000,4.210000,-5.260000,-3.870556e+04,3.290000,-0.833333,25.960524,30.485785,30.090741,0.000000,0.959450,1.511436,2.447328,1.576754,2.544076,1.199062,2.007103
75%,11.000000,0.000000,3600.0,9.206500,2.370000,-0.550000,3.730000,-1.065000,5.480000,-1.130000,7.455000,-1.530000,4.935000,-1.480000,5.710000,-2.020000,6.485000,-2.480000,9.107928e+04,13.045000,-0.787469,28.191843,33.439841,34.166085,0.134695,0.983881,9.805720,4.551434,9.963491,4.384304,8.095913,3.265921
max,19.000000,0.000000,3600.0,42602.150000,18.550000,1.710000,20.150000,1.080000,40.990000,9.590000,114.510000,10.530000,40.990000,1.080000,40.990000,0.930000,60.180000,0.930000,5.701408e+09,2453.090000,-0.750000,29.991608,50.000000,52.978440,0.924922,1.000000,40635.919279,14.007992,40556.219379,14.599309,2857.772845,13.130761


In [7]:
buy_dataframe.query("sarima_3_margin==sarima_3_margin").query("sarima_3_margin>4").describe()

,buy_score,candle_stick_score,granularity,price,one_hr_high,one_hr_low,three_hr_high,three_hr_low,three_six_high,three_six_low,six_12_high,six_12_low,six_hr_high,six_hr_low,nine_hr_high,nine_hr_low,twelve_hr_high,twelve_hr_low,obv,obv_pc,rri,rsi,rsi_last,rsi_prev,stochrsi14,williamsr14,sarima_1,sarima_1_margin,sarima_2,sarima_2_margin,sarima_3,sarima_3_margin
count,534.000000,534.000000,534.0,534.000000,534.000000,534.000000,534.000000,534.000000,519.000000,519.000000,511.000000,511.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,5.340000e+02,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000,534.000000
mean,12.142322,-0.018727,3600.0,11.447528,2.281423,-1.854532,3.938895,-2.916498,4.881079,-4.746224,5.427339,-6.723033,5.319401,-4.288745,6.306049,-5.310899,7.038521,-6.346067,-2.101135e+07,-2.777491,-0.866890,45.537009,54.284640,52.883131,0.136386,0.769641,12.059978,13.720198,12.089635,14.227831,12.118112,14.738223
std,3.866240,0.220066,0.0,66.724504,2.739231,1.656795,5.213727,2.557559,9.692223,4.987044,11.371620,7.297808,7.341631,3.552594,9.157001,4.144619,9.783051,5.025505,1.861725e+09,82.262741,0.086589,10.980876,12.645662,12.065895,0.208466,0.217795,70.457532,177.795815,70.724176,180.779256,70.822401,183.162216
min,4.000000,-2.000000,3600.0,0.000119,-0.760000,-10.270000,-0.450000,-23.880000,-11.190000,-24.660000,-10.130000,-31.440000,-0.450000,-23.880000,-0.450000,-24.660000,-0.450000,-27.480000,-4.044723e+10,-609.070000,-1.000000,15.179948,20.169277,21.690141,0.000000,0.230769,0.000124,0.025774,0.000124,0.491567,0.000124,4.002084
25%,9.000000,0.000000,3600.0,0.056370,0.882500,-2.537500,1.200000,-4.140000,0.355000,-7.660000,-0.610000,-10.810000,1.602500,-5.852500,1.895000,-7.540000,2.200000,-9.270000,-6.967627e+04,-10.367500,-0.925506,38.426397,45.815065,45.096489,0.000000,0.576968,0.060358,3.826758,0.059792,4.004312,0.059905,4.391277
50%,12.000000,0.000000,3600.0,0.283180,1.635000,-1.540000,2.370000,-2.315000,2.870000,-4.440000,3.240000,-5.830000,3.195000,-3.440000,3.655000,-4.555000,4.110000,-5.355000,6.497761e+05,-3.285000,-0.872232,46.936869,54.417657,52.634211,0.000000,0.837424,0.296156,4.747398,0.296014,5.076741,0.297993,5.265632
75%,14.000000,0.000000,3600.0,3.044300,2.725000,-0.612500,4.712500,-0.985000,5.900000,-1.500000,7.240000,-1.900000,6.012500,-1.622500,6.792500,-2.140000,7.370000,-2.395000,3.084365e+07,2.352500,-0.809790,53.836358,63.077253,61.225647,0.237018,0.971507,3.196439,6.382537,3.191314,6.766647,3.189003,6.929820
max,23.000000,1.000000,3600.0,1120.050000,36.000000,1.560000,58.250000,0.920000,87.020000,20.600000,87.020000,33.390000,83.440000,0.170000,87.020000,0.170000,87.020000,0.170000,2.430124e+09,1161.620000,-0.084661,73.104446,90.712825,89.605997,0.919776,1.000000,1185.908719,4103.256846,1192.976670,4173.272402,1192.055949,4230.002230


In [8]:
buy_dataframe.to_csv("buy_df_1hr_ema_rri_sarima_1236_rules.csv")

### Observation

**In the first hour,** 

- 50% of the BUY signals give 1.11% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.45% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour


- If invested Rs.100 and for 100 buy signals sellupprpcnt is 0.45, definite chance of making Rs.139 if 75% is succesful
- If invested Rs.100 and for 100 buy signals sellupprpcnt is 1.11, definite chance of making Rs.171 if 50% is succesful

- Filterd by ones that makes < 0.05 in first hour and six_hr_high is also less than 0.00 margin, there is none. Which means, even if it does. not make more than 0.05 margin in first hour, at some point it makes more than 0% profit

**In the three hours,**

- 50% of the BUY signals give 2.0% profit. Setting sellupprpcnt with 1% will get 50% success rate in first hour
- 75% of the buy signals give 0.8% profit. Setting sellupprpcnt with 0.4% will get 75% success rate in first hour

